In [1]:
import itertools as it

from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: 8e4c084ccf4e
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 2
nbname: postprocess-bias
nbpath: /opt/hstrat-evolutionary-inference/binder/postprocess-bias.ipynb
revision: null
session: 84e27cdb-b453-4dc7-894b-6ff906f792b5
timestamp: 2023-03-13T05:34:00Z00:00


IPython==7.16.1
keyname==0.5.2
yaml==6.0
matplotlib==3.5.3
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
seaborn==0.12.2
teeplot==0.2.0
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.0.3
json==2.0.9
six==1.16.0
ipykernel==5.5.3


In [3]:
df = pd.read_csv("https://osf.io/5trmg/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


3b0245309571602d


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,14,...,222245.143375,10.507880,0,24568,1222.888975,3.172174e+06,216613.033012,NaN,NaN,NaN
1,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,48,...,229130.407979,13.872127,0,27597,1218.312041,3.318682e+06,651239.465532,NaN,NaN,NaN
2,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,3,...,284849.005921,11.622811,0,25234,1211.424603,3.052790e+06,514176.080681,NaN,NaN,NaN
3,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,10,...,188583.333771,9.209171,0,25388,1292.644315,3.547016e+06,497751.272393,NaN,NaN,NaN
4,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,18,...,275538.079227,12.278440,0,24122,1293.437720,3.312494e+06,518201.905151,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85795,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,0,...,17.635489,17.635489,1,3036,170.558104,5.577250e+04,6234.641599,30.0,0.01,naive
85796,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,42,...,21.165177,21.165177,1,436594,2.072540,6.791300e+04,0.493756,3.0,1.00,rollback
85797,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,45,...,36.573624,36.573624,1,492068,3.755694,1.230666e+05,0.658304,10.0,1.00,expected
85798,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,4,...,36.416021,36.416021,1,3774,123.845566,4.049750e+04,2327.128776,30.0,0.01,rollback


In [4]:
# phylometrics whittled down to descriptive, non-redundant set
phylometrics = [
    #     'diversity',
    "colless_like_index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean_pairwise_distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    "num_ancestors",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean_evolutionary_distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [5]:
methodological_variables = [
    "resolution",
    "trie-postprocess",
]


In [6]:
evolutionary_variables = [
    "num_islands",
    "num_niches",
    "tournament_size",
]


In [7]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [8]:
fixed_variables = [
    "num_generations",  # homogeneous (num generations per epoch)
    "p_island_migration",  # homogeneous
    "population_size",  # homogeneous
    "p_niche_invasion",  # only use 3.0517578125e-08
    "subsampling-fraction",  # only use 1.0
]


In [9]:
# ensure fixed variables
df = df[
    (df["p_island_migration"] == 0.01)
    & (df["p_niche_invasion"] == 3.0517578125e-08)
    & ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1, excluded_variable

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,14,...,222245.143375,10.507880,0,24568,1222.888975,3.172174e+06,216613.033012,NaN,NaN,NaN
1,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,48,...,229130.407979,13.872127,0,27597,1218.312041,3.318682e+06,651239.465532,NaN,NaN,NaN
2,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,3,...,284849.005921,11.622811,0,25234,1211.424603,3.052790e+06,514176.080681,NaN,NaN,NaN
3,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,10,...,188583.333771,9.209171,0,25388,1292.644315,3.547016e+06,497751.272393,NaN,NaN,NaN
4,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,18,...,275538.079227,12.278440,0,24122,1293.437720,3.312494e+06,518201.905151,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85791,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,32,...,41.282080,41.282080,1,509221,2.442184,8.002550e+04,0.540384,10.0,1.0,rollback
85792,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,41,...,72.109885,72.109885,1,664279,4.229950,1.386070e+05,3.655409,30.0,1.0,naive
85794,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,42,...,99.885108,99.885108,1,740211,6.283763,2.059064e+05,5.657382,100.0,1.0,expected
85796,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,42,...,21.165177,21.165177,1,436594,2.072540,6.791300e+04,0.493756,3.0,1.0,rollback


In [10]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
300,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,31,...,19.400293,19.400293,0,534071,113.968277,3.734513e+06,2044.180574,30.0,1.0,naive
301,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,49,...,26.767984,26.767984,1,573215,89.236237,2.924093e+06,1971.137030,30.0,1.0,rollback
303,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,17.592952,17.592952,0,483414,118.163651,3.871987e+06,4409.976736,3.0,1.0,naive
305,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,11,...,22.991331,22.991331,0,531785,119.747513,3.923887e+06,9957.713723,10.0,1.0,naive
307,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,3,...,24.570888,24.570888,0,576970,122.424622,4.011610e+06,8435.151351,30.0,1.0,naive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85791,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,32,...,41.282080,41.282080,1,509221,2.442184,8.002550e+04,0.540384,10.0,1.0,rollback
85792,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,41,...,72.109885,72.109885,1,664279,4.229950,1.386070e+05,3.655409,30.0,1.0,naive
85794,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,42,...,99.885108,99.885108,1,740211,6.283763,2.059064e+05,5.657382,100.0,1.0,expected
85796,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,42,...,21.165177,21.165177,1,436594,2.072540,6.791300e+04,0.493756,3.0,1.0,rollback


In [11]:
df["num_niches"].unique()


array([4, 8, 1])

In [12]:
df["num_islands"].unique()


array([1024,    1])

In [13]:
df["tournament_size"].unique()


array([2, 8])

In [14]:
df["a"].unique()


array(['reconstructed-tree', 'collapsed-phylogeny'], dtype=object)

In [15]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2): "plain",
        (1, 1, 1): "neutral selection",
        (1, 1, 8): "strong selection",
        (1, 4, 2): "4 niche ecology",
        (1, 8, 2): "8 niche ecology",
        (1024, 1, 2): "spatial structure",
    }.get(
        tuple(row[["num_islands", "num_niches", "tournament_size"]]),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime"])
df["regime"].unique()


array(['8 niche ecology', 'spatial structure', '4 niche ecology',
       'strong selection', 'plain'], dtype=object)

In [16]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


array(['4 niche ecology', 'plain', 'strong selection',
       'spatial structure', '8 niche ecology'], dtype=object)

In [17]:
for setup, setup_df in df.groupby(sensitivity_analysis_variables):
    print(f"{setup=}")
    for regime, regime_df in setup_df.groupby(["regime"]):
        ground_truth = regime_df[(regime_df["a"] == "collapsed-phylogeny")][
            "average_origin_time_normalized"
        ].mean()
        print(f"   regime {ground_truth=}")

        for group, group_df in regime_df[
            (regime_df["a"] == "reconstructed-tree")
        ].groupby(["trie-postprocess", "resolution"]):
            err = (
                group_df["average_origin_time_normalized"].mean()
                - ground_truth
            )
            print("      ", group, len(group_df), err)


setup=(0, 'np.random.exponential')
   regime ground_truth=32764.960884426404
       ('expected', 3.0) 50 -0.03134534381388221
       ('expected', 10.0) 50 -0.0031501435114478227
       ('expected', 30.0) 50 -0.0006929588416824117
       ('expected', 100.0) 50 -0.0009169950535579119
       ('naive', 3.0) 50 -0.023581957459100522
       ('naive', 10.0) 50 0.0035981322653242387
       ('naive', 30.0) 50 0.005833918276039185
       ('naive', 100.0) 50 0.005579088716331171
       ('rollback', 3.0) 50 -0.028470412300521275
       ('rollback', 10.0) 50 -0.0003448591560299974
       ('rollback', 30.0) 50 0.0017712942935759202
       ('rollback', 100.0) 50 0.001644642477913294
   regime ground_truth=32762.279329203167
       ('expected', 3.0) 50 -0.062140025325788883
       ('expected', 10.0) 50 -0.00019378435536054894
       ('expected', 30.0) 50 -0.0009946209538611583
       ('expected', 100.0) 50 -0.0004676028802350629
       ('naive', 3.0) 50 -0.053548997471807525
       ('naive', 10.0) 50 